##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
#%pip install pandas 
#%pip install matplotlib
#%pip install numpy
#%pip install matplotlib
#%pip install xgboost
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .csv file inside that. A relative path *must* be used when loading data into pandas

In [ ]:
# Can have as many cells as you want for code
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 

filepath = "./data/catA_train.csv" 
df = pd.read_csv(filepath)
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

### 1. Exploring the Dataset

In [ ]:
df.head(3)

In [ ]:
print(df.shape)
df.describe()

In [ ]:
df.info()
df["Entity Type"].value_counts()

There are 29182 entries in this dataset and 28 columns, with a mix of numerical and categorical variable

In [ ]:
# Explore the number of missing values in the dataset 
df.isna().sum()

The features with missing values are: 
1. Latitude and Longitude
2. Year Found
3. Parent Company and Parent Country 
4. Square Footage
5. Employees (Single Site), Employees (Domestic Ultimate Total), Employees (Global Ultimate Total)
6. Import/Export Status
7. Fiscal Year End 
8. Global Ultimate Company and Global Ultimate Country 
9. Domestic Ultimate Company 

We will discuss how we handled these missing values in the next section.

### 2. Data Exploration and Visualisation


In [ ]:
industry_counts = df["Industry"].value_counts()
top_industries = industry_counts.head(5)
other_count = industry_counts[5:].sum()
top_industries['Other'] = other_count
plt.figure(figsize=(6,6))
plt.pie(top_industries, autopct='%1.1f%%', startangle=140,textprops={"fontsize" : 8})
plt.title('Number of Companies by Industry (Top 5 Categories)')
plt.legend(top_industries.index, title="Industries", loc="lower left", fontsize = 6)



We can observe from the pie chart that the most common industry the companies are involved in are Offices of Holding Companies, we have also observed 582 unique industries in the dataset. As such, we will be trying to condense these categories further 

In [ ]:
#Creating a scatter plot of 
plt.figure(figsize=(10, 6))
plt.scatter(df['Sales (Global Ultimate Total USD)'], df['Sales (Domestic Ultimate Total USD)'])
plt.title('Correlation between Global and Domestic Sales')
plt.xlabel('Global Sales') 
plt.ylabel('Domestic Sales')  


# Creating a scatter plot of lg domeestic sales vs lg globalsales
plt.figure(figsize=(10, 6))
plt.scatter(np.log(df['Sales (Global Ultimate Total USD)']), np.log(df['Sales (Domestic Ultimate Total USD)']))
plt.title('Correlation between log Global and log Domestic Sales')
plt.xlabel('log Global Sales')  
plt.ylabel('log Domestic Sales') 

We explored the relationship between the Global Ultimate Sales and Domestic Ultimate Sales. In our initial plot, due to large scale of the numbers, we were unable to determine any relationship between the two features.

We explored transforming the features and could observe that there was some correlation between the log-transformed variables.

In [ ]:
#Scatterplot of global employee size and lg domestic sales
plt.figure(figsize=(10, 6))
plt.scatter(df['Employees (Global Ultimate Total)'], np.log(df['Sales (Domestic Ultimate Total USD)']))
plt.title('Correlation between Employees(Global Ultimate Total) and Domestic Sales')
plt.xlabel('Employees(Domestic )')  # x-axis label
plt.ylabel('Domestic Sales')  # y-axis label

#Scatterplot of domestic employee size and lg domestic sales
plt.figure(figsize=(10, 6))
plt.scatter(df['Employees (Domestic Ultimate Total)'], np.log(df['Sales (Domestic Ultimate Total USD)']))
plt.title('Correlation between Employees (Domestic Ultimate Total) and Domestic Sales')
plt.xlabel('Employees(Global)')  # x-axis label
plt.ylabel('Domestic Sales')  # y-axis label

We can see that there is some relationship between the size of the global and domestic ultiate workforce. We plan to investigate this relationship further in our modelling. 

In [ ]:
industry_sales = df.groupby('Industry')['Sales (Domestic Ultimate Total USD)'].mean()
top_industries = industry_sales.sort_values(ascending=False).head(5)
plt.figure(figsize=(10, 6))  # Adjust the size as needed
top_industries.plot(kind='barh')
plt.title('Top 5 Industries by Average Sales')
plt.xlabel('Average Total Sales')
plt.ylabel('Industries')

# Show the plot
plt.show()

Analysing the avergae sales by industries, we can observe that certain industries tend to have a significantly higher average sales than other industries, thus we are planning to explore the effect of industries on the domestic sales. 

In [ ]:
industry_sales = df.groupby('Global Ultimate Country')['Sales (Domestic Ultimate Total USD)'].mean()
top_industries = industry_sales.sort_values(ascending=False).head(5)
plt.figure(figsize=(10, 6))  # Adjust the size as needed
top_industries.plot(kind='barh')
plt.title('Top 5 Global Ultimate Countries by Average Sales')
plt.ylabel('Global Ultimate Countries')
plt.xlabel('Average Total Sales')

# Show the plot
plt.show()

Analysing the avergae sales by Global Ultimate Countries, we can observe that certain Countries tend to have a significantly higher average sales than other countries, thus we are planning to explore the effect of countries on the domestic sales. 

In [ ]:
industry_sales = df.groupby('Parent Country')['Sales (Domestic Ultimate Total USD)'].mean()
top_industries = industry_sales.sort_values(ascending=False).head(5)
plt.figure(figsize=(10, 6))  # Adjust the size as needed
top_industries.plot(kind='barh')
plt.title('Top 5 Parent Countries by Average Sales')
plt.ylabel('Parent Country')
plt.xlabel('Average Total Sales')

# Show the plot
plt.show()

Analysing the average sales by Parent Countries, we can observe that certain Countries tend to have a significantly higher average sales than other countries, thus we are planning to explore the effect of Parent countries on the domestic sales. 

In [ ]:
industry_sales = df.groupby('Import/Export Status')['Sales (Domestic Ultimate Total USD)'].mean()
top_industries = industry_sales.sort_values(ascending=False).head(5)
plt.figure(figsize=(10, 6))  # Adjust the size as needed
top_industries.plot(kind='barh')
plt.title('Top 5 Parent Countries by Average Sales')
plt.ylabel('Import/Export Status')
plt.xlabel('Average Total Sales')

# Show the plot
plt.show()

Looking at the aveage sales for import/export type, we find that companies involved in both import and export, as well as those with missing values for this column share a relatiely higher average total domestic sales, we plan too further investigate this in our model.

In [ ]:
industry_counts = df["Industry"].value_counts()
top_industries = industry_counts.head(5)
other_count = industry_counts[5:].sum()
top_industries['Other'] = other_count
plt.figure(figsize=(6,6))
plt.pie(top_industries, autopct='%1.1f%%', startangle=140,textprops={"fontsize" : 8})
plt.title('Number of Companies by Industry (Top 5 Categories)')
plt.legend(top_industries.index, title="Industries", loc="lower left", fontsize = 6)



### 3. Handling of Missing Values

Latitude and Longitude: we believe that the company location could be a significant feature in training the model, hence we decided to remove rows with missing values in these columns.

In [ ]:
# remove rows with na values in longitude and latitude columns, as well as having negative sales
df = df.dropna(subset=["LONGITUDE", "LATITUDE"])

Year Found: we decided to fill the missing values with mode since the other two possible imputers - median and mean seems to be rather high and we think that mode would be the more apporpriate imputer. 

In [ ]:
df["Year Found"] = df["Year Found"].fillna(value = df["Year Found"].mode()[0])

Parent Country & Global Ultimate Country: We identified them to be potential important features. Therefore, for missing values in these two features, we classify the missing values as "Missing". However, we do not consider the exact company itself as that might lead to overfitting of the model, that includes Parent Company, Global Ultimate Company and Domestic Ultimate Company.

In [ ]:
toReplace = ["Parent Country", "Global Ultimate Country"]
df[toReplace] = df[toReplace].fillna(value = "Missing")

Fiscal Year End & Square Footage: these column have a lot of (or all) missing values and we think that this feature is not so meaningful, hence we decided to drop this feature entirely.

Employees (Single Site, Domestic Ultimate Total, Global Ultimate Total): Since there are many missing values for Employee (Single Site), we decided to drop this feature entirely and utilized on Employees (Domestic Ultimate Total) as well as Employes (Global Ultimate Total). The missing values for the latter two are filled with the mean imputers. 

In [ ]:
df["Employees (Domestic Ultimate Total)"] = df["Employees (Domestic Ultimate Total)"].fillna(df["Employees (Domestic Ultimate Total)"].mean().round())
df["Employees (Global Ultimate Total)"] = df["Employees (Global Ultimate Total)"].fillna(df["Employees (Global Ultimate Total)"].mean().round())

Import/Export Status: lastly, we believe this is important as this data sort of tells us the scale of the company. Therefore, we classify missing values as "Missing", same as what we did for Parent Country and Global Ultimate Country.

In [ ]:
df["Import/Export Status"] = df["Import/Export Status"].fillna(value = "Missing")

### 3. Feature Engineering


Firstly, there are some negative sales data in the dataset and we decided to drop these rows. We also only consider active companies. 

In [ ]:
df = df.drop(df.index[df["Sales (Domestic Ultimate Total USD)"] < 0])
df = df.drop(df.index[df["Sales (Global Ultimate Total USD)"] < 0])
df = df.drop(df.index[df["Company Status (Active/Inactive)"] == "Inactive"])

Next, we think that it might be easier to work with the feature "Age" rather than "Year Found", since "Age" can be considered as a numerical variable. 

In [ ]:
df["Age"] = pd.to_datetime("today").year - df["Year Found"] 

There are many unique SIC codes in the dataset (582) which may lead to overfitting. Therefore, we found a relatively good guide online that helps us condense these SIC codes into 10 main industries. We will create a new column named "Industry Type". 

In [ ]:
def classifySICcode(SIC):
    if SIC//100 <= 9 and SIC//100 >= 1 : 
        return "Agriculture, Forestry, Fishing"
    elif SIC//100 <= 14 and SIC//100 >= 10 : 
        return "Mining"
    elif SIC//100 <= 17 and SIC//100 >= 15 : 
        return "Construction"
    elif SIC//100 <= 39 and SIC//100 >= 20:
        return "Manufacturing"
    elif SIC//100 <= 49 and SIC//100 >= 40:
        return "Transportation & Public Utilities"
    elif SIC//100 <= 51 and SIC//100 >= 50:
        return "Wholesale Trade"
    elif SIC//100 <= 59 and SIC//100 >= 52:
        return "Retail Trade"
    elif SIC//100 <= 67 and SIC//100 >= 60:
        return "Finance, Insurance, Real Estate"
    elif SIC//100 <= 89 and SIC//100 >= 70:
        return "Services"
    elif SIC//100 <= 99 and SIC//100 >= 91 : 
        return "Public Administration"
    else:
        return "Other" 
    
df["Industry Type"] = df["SIC Code"].transform(lambda x:classifySICcode(x))

In [ ]:
industry_counts = df["Industry Type"].value_counts()
top_industries = industry_counts.head(5)
other_count = industry_counts[5:].sum()
top_industries['Other'] = other_count
plt.figure(figsize=(6,6))
plt.pie(top_industries, autopct='%1.1f%%', startangle=140,textprops={"fontsize" : 6})
plt.title('Number of Companies by Industry Type')
plt.legend(top_industries.index, title="Industries", loc="lower left", fontsize = 6)

By further categorising it into 10 smaller subgroups, we are able to gain a better understanding of the dataset and industries involved

To understand the longitude and latitude data better, we try to group companies into different clusters using KMeans clustering and give them a label (a new column named "Location Label"). We choose the number of clusters to be 6 since Singapore has 6 large industrial parks spread across the Republic.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

# KMeans to cluster companies
X = df[["LATITUDE" , "LONGITUDE"]]
kmeans = KMeans(n_clusters = 6, n_init="auto").fit(X)
df["Location Label"] = kmeans.labels_

# training KNN Clustering model to be used for new data 
cluster_centers = kmeans.cluster_centers_
labels = range(kmeans.n_clusters)

knn = KNeighborsClassifier(n_neighbors=1)  # Using 1 neighbor for exact match
knn.fit(cluster_centers, labels)

# Simple visualization of the cluster
plt.scatter(df["LONGITUDE"], df["LATITUDE"], c = df["Location Label"])


For the features "Parent Country" and "Global Ultimate Country", if there are less than 10 occurences, we will classify them as "Others" instead to reduce the dimension of the dataset. 

In [ ]:
freq_parent = df["Parent Country"].value_counts() 
low_freq_parent_countries = freq_parent[freq_parent < 10].index
df["Parent Country"] = df["Parent Country"].replace(low_freq_parent_countries, "Others")

# Do the same for Global Ultimate Country
freq_global_ultimate = df["Global Ultimate Country"].value_counts() 
low_freq_global_ultimate_countries = freq_global_ultimate[freq_global_ultimate < 10].index
df["Global Ultimate Country"] = df["Global Ultimate Country"].replace(low_freq_global_ultimate_countries, "Others")


We will also transform all categorical variables into indicator variables for model training purposes. We also drop irrelevant columns at this stage.

In [ ]:
col_to_keep = ["Sales (Domestic Ultimate Total USD)", "Sales (Global Ultimate Total USD)", "Parent Country", 
               "Employees (Domestic Ultimate Total)", "Employees (Global Ultimate Total)", "Import/Export Status", 
               "Global Ultimate Country", "Is Domestic Ultimate", "Is Global Ultimate", "Age", "Location Label", 
               "Industry Type", "Ownership Type"]
df = df[col_to_keep]

col_to_encode = ["Parent Country", "Import/Export Status", "Global Ultimate Country", "Is Domestic Ultimate", 
                 "Is Global Ultimate", "Location Label", "Industry Type", "Ownership Type"]
for var in col_to_encode:
    encoded = pd.get_dummies(df[var], prefix = var)
    df = df.drop(var, axis = 1)
    df = pd.concat([df, encoded], axis = 1)

df.head(3)


### 4. Model Building

We will try to use one of the most robust and accurate algorithm (XGBoost) to build our predictive model. For evaluation metrics, we will be using R^2 values. We will also be conducting cross validation using the k-fold method.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

# We first separate our features and target variable
X = df.drop(["Sales (Domestic Ultimate Total USD)"], axis = 1)
y = df["Sales (Domestic Ultimate Total USD)"]

# Initialize model 
model = XGBRegressor(n_estimators = 239, max_depth = 5, learning_rate = 0.18)

for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    # Use 20-fold 
    kf = KFold(n_splits = 20, shuffle = True)
    cv_scores = cross_val_score(model, X_train, y_train, cv = kf, scoring = "r2")

    print("CV R^2 errors: ", cv_scores)
    print("Mean R^2 errors: ", cv_scores.mean())

    # Train and evaluate 
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    print("R-squared error on test set: ", r2_score(y_test, y_pred_test))


As we can see from the performance above, generally, the model performs quite well in terms of r-squared value. The final hyperparameters chosen for the XGBoostRegressor is based on multiple trials and errors with some help on the library Optuna. 

In [ ]:
from xgboost import plot_importance

feature_importance = model.feature_importances_
index = feature_importance.argsort()[::-1]

# Simple Visualization
plot_importance(model, max_num_features = 10, importance_type = "cover")
plot_importance(model, max_num_features = 10, importance_type = "weight")
plot_importance(model, max_num_features = 10, importance_type = "gain")


There are 3 Methods of Computing Feature Importance:
- Weight: The number of times a feature appears in a tree across the ensemble of trees.
- Gain: The average gain of a feature when it is used in trees.
- Cover: The average coverage of a feature when it is used in trees.


Based on the chart for the weight of feature importance, it seems that most of features that we exlpored in our data analysis have also shown up on the chart. 

### 5. Final Model

Finally, we fit all data in the dataset using the final parameters we have and save the final model as BestModel.

In [ ]:
from xgboost import XGBRegressor
import joblib

# We first separate our features and target variable
X = df.drop(["Sales (Domestic Ultimate Total USD)"], axis = 1)
y = df["Sales (Domestic Ultimate Total USD)"]

# Initialize model 
BestModel = XGBRegressor(n_estimators = 239, max_depth = 5, learning_rate = 0.18)
BestModel.fit(X, y)
joblib.dump(BestModel, "FinalModel.h5")


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''

    df = hidden_data
    # fill missing values of longitude and latitude with mean 
    df["LATITUDE"] = df["LATITUDE"].fillna(value = df["LATITUDE"].mean())
    df["LONGITUDE"] = df["LONGITUDE"].fillna(value = df["LONGITUDE"].mean())
    df["Location Label"] = knn.predict(df[["LATITUDE", "LONGITUDE"]])

    # fill missing values of years with mode 
    df["Year Found"] = df["Year Found"].fillna(value = df["Year Found"].mode()[0])
    df["Age"] = pd.to_datetime("today").year - df["Year Found"]

    # fill missing values of global sales with mode 
    df["Sales (Global Ultimate Total USD)"] = df["Sales (Global Ultimate Total USD)"].fillna(value = df["Sales (Global Ultimate Total USD)"].mean())

    # Convert SIC code to industry type 
    def classifySICcode(SIC):
        if SIC//100 <= 9 and SIC//100 >= 1 : 
            return "Agriculture, Forestry, Fishing"
        elif SIC//100 <= 14 and SIC//100 >= 10 : 
            return "Mining"
        elif SIC//100 <= 17 and SIC//100 >= 15 : 
            return "Construction"
        elif SIC//100 <= 39 and SIC//100 >= 20:
            return "Manufacturing"
        elif SIC//100 <= 49 and SIC//100 >= 40:
            return "Transportation & Public Utilities"
        elif SIC//100 <= 51 and SIC//100 >= 50:
            return "Wholesale Trade"
        elif SIC//100 <= 59 and SIC//100 >= 52:
            return "Retail Trade"
        elif SIC//100 <= 67 and SIC//100 >= 60:
            return "Finance, Insurance, Real Estate"
        elif SIC//100 <= 89 and SIC//100 >= 70:
            return "Services"
        elif SIC//100 <= 99 and SIC//100 >= 91 : 
            return "Public Administration"
        else:
            return "Other" 
    
    df["SIC Code"] = df["SIC Code"].fillna(df["SIC Code"].mode()[0])
    df["Industry Type"] = df["SIC Code"].transform(lambda x:classifySICcode(x))

    # Fill missing values for employees
    df["Employees (Domestic Ultimate Total)"] = df["Employees (Domestic Ultimate Total)"].fillna(df["Employees (Domestic Ultimate Total)"].mean().round())
    df["Employees (Global Ultimate Total)"] = df["Employees (Global Ultimate Total)"].fillna(df["Employees (Global Ultimate Total)"].mean().round())

    # Fill NA Import/Export Status with "Missing"
    df["Import/Export Status"] = df["Import/Export Status"].fillna(value = "Missing")

    # Replace missing Country with "Missing"
    toReplace = ["Parent Country", "Global Ultimate Country"]
    df[toReplace] = df[toReplace].fillna(value = "Missing")

    # Classify new countries as Others
    GlobalCountries = ['United Kingdom', 'Singapore', 'Hong Kong SAR', 'France', 'Germany',
 'Cayman Islands' ,'Switzerland', 'Norway', 'Spain', 'Denmark', 'India',
 'Curacao' ,'Missing' ,'United States', 'Japan', 'China', 'Luxembourg',
 'Virgin Islands (British)', 'Panama', 'Netherlands' ,'Italy', 'Malaysia',
 'Australia' ,'Korea, Republic of' ,'Canada', 'Taiwan' ,'Others' ,'Vietnam',
 'Indonesia', 'Sweden', 'United Arab Emirates', 'Portugal' ,'New Zealand',
 'Ireland' ,'Finland' ,'Thailand' ,'Austria', 'Belgium' ,'Bermuda',
 'South Africa' ,'Brazil' ,'Mauritius' ,'Israel', 'Cyprus', 'Philippines',
 'Liechtenstein']
    
    ParentCountries = ['Singapore' ,'Hong Kong SAR', 'Netherlands', 'Cayman Islands' ,'Luxembourg',
 'Switzerland' ,'Spain' ,'Denmark' ,'India', 'Missing' ,'United Kingdom',
 'Japan' ,'United States' ,'China' ,'Germany', 'Virgin Islands (British)',
 'Malaysia', 'Korea, Republic of', 'France', 'Australia', 'Taiwan', 'Norway',
 'Others' ,'Bermuda', 'Ireland' ,'Canada' ,'Vietnam', 'Indonesia', 'Sweden',
 'United Arab Emirates' ,'Cyprus', 'New Zealand' ,'Thailand' ,'Austria',
 'Israel' ,'Belgium' ,'Italy', 'Philippines', 'Malta' ,'Finland', 'Mauritius',
 'Panama']
    
    df["Parent Country"] = df["Parent Country"].where(df["Parent Country"].isin(ParentCountries), "Others")
    df["Global Ultimate Country"] = df["Global Ultimate Country"].where(df["Global Ultimate Country"].isin(GlobalCountries), "Others")
    
    # fill missing values of ownership type with mode
    df["Ownership Type"] = df["Ownership Type"].fillna(value = df["Ownership Type"].mode()[0])

    # fill is domestic/global ultimate with mode 
    df["Is Domestic Ultimate"] = df["Is Domestic Ultimate"].fillna(value = df["Is Domestic Ultimate"].mode()[0])
    df["Is Global Ultimate"] = df["Is Global Ultimate"].fillna(value = df["Is Global Ultimate"].mode()[0])

    # Drop irrelevant columns
    col_to_keep = ["Sales (Global Ultimate Total USD)", "Parent Country", 
               "Employees (Domestic Ultimate Total)", "Employees (Global Ultimate Total)", "Import/Export Status", 
               "Global Ultimate Country", "Is Domestic Ultimate", "Is Global Ultimate", "Age", "Location Label", 
               "Industry Type", "Ownership Type"]
    df = df[col_to_keep]

    col_to_encode = ["Parent Country", "Import/Export Status", "Global Ultimate Country", "Is Domestic Ultimate", 
                 "Is Global Ultimate", "Location Label", "Industry Type", "Ownership Type"]
    
    
    for var in col_to_encode:
        encoded = pd.get_dummies(df[var], prefix = var)
        df = df.drop(var, axis = 1)
        df = pd.concat([df, encoded], axis = 1)

    loaded_model = joblib.load('./FinalModel.h5')
    result = loaded_model.predict(df)
    

    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
filepath = "./data/catA_train.csv" 
test_df = pd.read_csv(filepath)
y = test_df['Sales (Domestic Ultimate Total USD)']
test_df = test_df.drop(columns=['Sales (Domestic Ultimate Total USD)'])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!